> ### Outline

1. **언어 모델**
    - 통계기반 모델
        - SLM
        - N-gram
        - PPL
    - 딥러닝기반 모델
        - BERT
            - 특징
            - 사전학습(1) : Masked Language Model(MLM)
            - 사전학습(2) : Next Sentence Prediction(NSP)
        - GPT
            - LLM의 한계
        - RAG
            - RAG의 구조
        - Langchain
            - Langchain의 아이디어, 특징
    - sLM
        - 모델 압축
        - sLM의 장단점
        - sLM과 LLM의 결합
2. **생성 모델**
    - 분류모델과 생성모델의 차이
    - 생성모델의 분류
        - AE
        - VAE
        - GAN
            - 구조와 장단점
        - 확산모델
            - 순확산과 역확산

---

> ## 1. 언어 모델

언어 모델은 단어 시퀀스(문장)에 확률을 할당 -> 가장 자연스러운 단어 시퀀스를 찾는 모델!  
'문장의 자연스러움'을 판단하는 가장 보편적인 방법  
    - 이전 단어가 주어졌을 때, 다음 단어 중 가장 높은 확률의 단어를 제시하는 방법


- 통계적 언어 모델 & 딥러닝적 언어 모델

|  | 핵심 원리 | 계산 복잡도 | 예시 |
| --- | --- | --- | --- |
| **통계 기반** | 문장의 등장 빈도 기반 -> **확률**을 계산 | 상대적으로 낮고, 학습 속도가 빠름 | N-gram, Perplexity  |
| **인공 신경망 기반** | 단어를 벡터로 표현 -> **신경망을 이용**해 다음 단어 예측 | 높고, 학습에 많은 시간과 컴퓨팅 자원이 필요 | RNN, LSTM, GRU, Transformer (BERT, GPT 등) |

<br>

> ### 1-1. 통계적 언어 모델(SLM)

**'분포 가설'이라는 아이디어에 근거하는 언어 모델**

- 분포 가설이란?  
"비슷한 문맥에서 함께 나타나는 단어들은 비슷한 의미를 가진다"는 가설  
= 어떤 단어(A)의 주변에서 특히 B,C,D라는 단어가 많이 등장한다면, B,C,D의 의미를 짐작할 수 있다.

분포 가설을 설명해주는 기본 개념

1. 조건부 확률  
$$P(A \mid B) = \frac{P(A , B)}{P(B)}$$

a.  이 단어에서 다음 단어의 확률을 순차적으로 계산  
    b. a에서 구한 각각의 확률을 모두 곱함 -> 문장 전체의 최종 확률  
    c. b의 최종 확률이 높을수록 그 문장은 더 자연스러움!  
> 예시

"나는 귤을 먹었다"라는 문장의 확률 =  
(문장 처음이 '나는'일 확률)X('나는' 다음 '귤을'이 나올 확률)X('귤을' 다음 '먹었다'가 나올 확률)  
$$P(나는, 귤을, 먹었다) = P(나는) \times P(귤을 \mid 나는) \times P(먹었다 \mid 나는, 귤을)$$

2. 카운트 기반

이전 단어 시퀀스의 등장 **빈도**를 통해 다음 단어의 확률 계산  

'전체 학습 데이터' 중에서 조건부 확률을 구함  
> 예시

전체 데이터에서 '나는 귤을 먹었다'의 등장 횟수:93 '나는 귤을'의 등장 횟수:100일 경우 확률은 93%  

- 카운트 기반 언어 모델의 한계: 희소 문제  
    학습 데이터의 부족으로 언어를 정확하게 모델링하지 못하는 문제.  
    현실에서 자연스러운 표현이지만 학습 데이터에 없을 경우, 확률을 판단하지 못함  
    학습 데이터에 없는 문맥 또한 확률을 판단하지 못함

> ### 1-2. 대표적인 SLM  

> N-gram  
기본은 카운트 기반 SLM이지만, 바로 앞 단어 전체를 참고하던 기존 방식과 다르게, **일부 단어만 참고**하는 방식 -> **희소 문제를 완화**하는 아이디어  


- 이유  
    앞 예시에서, 문장이 길어질수록(나는 귤을 먹었다 -> 나는 시큼한 귤을 먹었다) 참고하는 단어가 많아지고, 만약 '시큼한 귤을 먹었다'가 코퍼스에 없다면 확률 계산이 불가해진다.  
- N-gram의 방식  
    '시큼한 귤을 먹었다'는 없어도 '시큼한 귤'을 코퍼스에서 잡아낼 가능성이 더 높아진다.

-> 참고하는 단어의 범위를 N개로 정해놓고 확률을 계산한다!

N의 값 | 이름 | 참고하는 단어 수 | 예시
------------------|----------------|--------------------------|----------------------------------------
1 | Unigram | 0개 (자기 자신) | $P(\text{먹었다})$
2 | Bigram | 1개 (바로 앞 1개) | $P(\text{먹었다} \mid \text{귤을})$
3 | Trigram | 2개 (바로 앞 2개) | $P(\text{먹었다} \mid \text{시큼한 귤을})$
N | N-gram | N-1개 |

- N-gram은 희소 문제를 완화했지만, 여전히 한계가 존재한다.  
    1. 여전한 희소 문제  
    위 예시에서, 2-gram을 사용하더라도 코퍼스에 '시큼한 귤'이란 표현이 없다면 계산 불가  
    2. n을 선택할 때의 문제  
    n이 큼 -> 더 많은 문맥을 고려 가능하지만 희소 문제 발생  
    n이 작음 -> 희소 문제는 덜하지만 문맥 이해도가 떨어짐  
    (일반적으로 n이 5를 넘지 않는 것을 권장)  
    3. 문맥 파악의 한계  
    중간의 수식어 등으로 중요한 단어 사이가 떨어져 있을 경우, 문맥을 파악하지 못하는 문제  

> Perlexity(PPL)  
테스트 데이터를 통해 언어 모델의 성능을 평가하기 위한 지표

언어 모델이 특정 문장을 얼마나 혼란스러워하는지를 측정. 즉, 낮을수록 좋음!
- 수식  
W: 평가하려는 문장  
N: 문장 W에 포함된 단어의 촐 개수  
P(W): 언어 모델이 문장 W에 부여하는 확률  
$$PPL(W) = P(w_1, w_2, w_3, ... , w_N)^\frac{-1}{N}$$

- PPL에 영향을 주는 요인  
W의 희소성 -> 희소하면 훈련 데이터에서 적게 봄 -> PPL 증가  
N의 개수(=문장의 길이) -> 함께 보는 단어가 많아짐 -> PPL 감소

> SLM의 한계와 딥러닝 기반 언어 모델의 필요성

- N-gram은 통계적 언어 모델의 한계를 일부 개선했지만, 여전히 희소 문제와 문맥 파악의 한계를 극복하지 못함  
- SLM은 훈련 데이터에 없는 문장이 나타나면 크게 혼란스러워하며 PPL 수치가 높게 나올 수밖에 없음  
- 반면, **딥러닝 기반의 언어 모델**은 단어와 문맥 간의 관계를 학습하여 일반화하는 능력이 뛰어남 -> 훈련 데이터에서 직접 보지 못한 문장이 등장하더라도 훨씬 유연하게 대처할 수 있음!



---

> ### 1-3. 딥러닝 기반 언어 모델

> LLM(Large Language Model)

방대한 양(Large)의 데이터를 학습한 언어 모델  
SLM(Small): 제한된 양의 텍스트 학습. 작은 규모이고 빠름  
NLM(Neural): 인공신경망 기반으로, 임베딩, 문장 완성, 기계 번역 등의 NLP에 활용  
PLM(Pre-Trained): 대규모의 데이터로 미리 학습된 언어 모델, 전이학습을 통해 NLP에 활용  

- LLM의 작동 방식  
1. 학습  
수십억 개의 문장으로 이루어진 방대한 데이터를 학습  
단어와 단어 사이의 관계, 문맥적 의이, 문법, 지식까지 언어의 패턴을 스스로 학습

2. 예측 및 생성  
사용자의 지시 -> 1에서 학습한 패턴을 기반으로, 다음에 올 가장 확률 높은 단어를 순서대로 예측하며 응답 생성

> BERT
텍스트의 앞뒤 문맥을 동시에 고려(Bidirectional), 입력값을 벡터로 바꾸는(Encoder), 딥러닝 모델(Transformer)  
-> 트랜스포머의 인코더 부분을 활용해 만든 양방향 언어 모델

- BERT의 특징  
1. 양방향: 문장을 읽을 때 앞뒤를 동시에 고려(앞뒤의 문맥을 동시에 참조)  
2. 트랜스포머 구조의 인코더 부분만 사용(GPT는 디코더 부분만 사용)  
3. 사전 학습: MLM(Masked)과 NSP(Next Sentence Prediction)를 통해 사전 학습  
4. 파인 튜닝: 학습 이후 특정 과제에 맞게 살짝 조정

- BERT의 학습 과정  
구조는 **트랜스포머의 인코더를 쌓아 올린** 구조  
Base는 12개, Large는 24개를 사용  

    -처리 과정: 인풋 -> 임베딩 -> 트랜스포머 인코더들 -> 아웃풋  

    - 임베딩: BERT는 세 가지의 임베딩을 합쳐서 사용하는데,  
    토큰 임베딩: 단어 자체의 의미  
    세그먼트 임베딩: 두개 이상의 문장이 들어올 때 어떤 문장인지 구분  
    포지션 임베딩: 단어의 문장 내 위치 정보를 포함  
    -> 세 가지 정보를 더해서 하나의 입력 벡터로 만듦

    - 사전 학습(MLM)  
    빈칸 채우기 퀴즈 같은 형식을 통해 문맥 추론 능력을 학습  
    예시 문장) 나는 오늘 [아침]에 사과를 먹었다  

| **마스킹 처리 방식** | **비율** | **예시** | **이유** |
| --- | --- | --- | --- |
| [MASK]로 바꾸기 | 80% | [MASK] | 모델이 가려진 단어를 맞추도록 훈련 |
| 랜덤 단어로 바꾸기 | 10% | 아침 → 연필 | 모델이 [MASK]가 아니어도 추측 가능하게 |
| 그대로 두기 | 10% | [아침] 유지 | 정답 단어를 보는 경우도 훈련에 포함 |

- -> 이렇게 다양한 상황을 줘서 **모델이 너무 [MASK]에만 의존하지 않도록 한다**  
- -> 이후 **양방향 학습**을 통해 **[MASK] 되어 있지 않은 부분에 대해서도 학습**

    - 사전 학습(NSP)  
    두 문장이 실제로 이어지는지에 대해 맞히는 학습  
    이어지는 문장(IsNext), 무작위 문장(NotNext)을 구분하는 학습

- BERT 이후의 모델  
    - RoBERTa  
    더 큰 데이터셋으로 많이 학습  
    NSP 과정 제거  
    배치 크기, 학습률 최적화  
    마스킹 패턴을 동적으로 변경 -> 다양한 패턴 학습  

    - ALBERT  
    레이어 간 파라미터 공유 -> 모델 크기 감소  
    임베딩 행렬을 두개의 작은 행렬로 분해 -> 더 가볍게

> GPT
Generative Pre-trained Transformer  
BERT가 인코더만을 사용한 모델이었다면, GPT는 디코더만을 사용하는 모델

- GPT의 구조(GPT-2의 아키텍쳐를 표준으로 사용)  
![GPT2](https://miro.medium.com/v2/resize:fit:1400/1*Ez3XmeCXcEgFYtm-DgzmZg.png)
    - 트랜스포머의 디코더만 사용  
    - 디코더의 셀프 어텐션은 Masked 어텐션으로 동작  
    미래(오른쪽) 단어를 볼 수 없게 마스킹 처리

- GPT의 동작 원리  
    - 입력된 문장의 앞 단어들을 기반으로, 다음 단어들의 확률 분포 계산  
    - 확률이 가장 높은 단어를 선택하면서 문장을 순차적으로 생성(AutoRegressive)

- BERT와의 차이  
    - BERT: 문장 **전체**를 보고 빈칸을 맞추는 방식 -> 양방향, 이해 중심  
    - GPT: 앞 단어들을 보고 다음 단어를 생성하는 방식 -> 단방향, 생성 중심  

![비교](https://blog.kakaocdn.net/dna/Aea5Z/btsgxoyOcMB/AAAAAAAAAAAAAAAAAAAAAFQxuii3zQJENTH6tcmwubXJpWFh9tM_eTX6yYZzfjwE/img.png?credential=yqXZFxpELC7KVnFOS48ylbz2pIh7yKj8&expires=1764514799&allow_ip=&allow_referer=&signature=OMQ6eabQXJX29RfgUgu3KraN5Gs%3D)
---

> 환각(할루시) 문제

아무리 많은 데이터를 학습시켜도, 특정 도메인이나 너무 지엽적인 지식을 모를 수 있음.. 따라서 그럴듯하게 지어내는 현상이 발생함  
LLM을 새로 학습 시키는 데에도 천문학적인 비용이 발생하므로, 대안으로 등장한 것이 RAG 기반!

> RAG  
Retrieval Augmented Generation
관련된 정보를 검색하여 먼저 찾고, 이를 LLM과 함께 최종 답변을 생성할 수 있는 하나의 파이프라인!

- RAG의 구조  
    1. 질의 인코더(Query Encoder)  
    사용자의 **질문을 이해**하기 위한 언어 모델  
    쿼리를 벡터 형태로 인코딩  
    2. 지식 검색기(Knowledge Retriever)  
    1에서 인코딩된 질문을 바탕으로, 외부 지식 베이스에서 **검색**  
    3. 지식 증강 생성기(Knowledge-Augmented Generator)  
    2에서 검색된 지식을 활용해 **답변을 생성**하는 언어 모델

- RAG는 환각을 방지하는 데 효과적이나, 검색된 문서의 신뢰성이 낮거나 잘못된 정보일 경우 답변의 정확도가 떨어질 수 있음.  
-> 이를 보완하기 위해 CAG 등장

> CAG  
Credibility-Aware Generation

- RAG의 한계를 보완하기 위해, 컨텍스트가 모델에 미치는 부정적 영향을 최소화  
- 모델이 문서의 신뢰도를 **스스로 판단**하고 활용할 수 있도록 훈련시키는 기법

> LangChain  
LLM+RAG의 파이프라인을 쉽게 구성할 수 있도록 도와주는 프레임워크!

- LangChain의 특징
    1. 추상화  
    각종 작업들을 한 줄 짜리 컴포넌트로 제공
    2. 표준화  
    비슷한 기능을 갖추고 있는 요소들을, 똑같은 형식으로 통일화  
    AI모델, 문서에 모두 표준화가 적용됨  
    3. 체이닝  
    컴포넌트들을 쉽게 연결 가능
    ```
    chain = prompt_template | ChatOpenAI() | StrOutputParser()
    chain.invoke({"language": "영어", "text": "안녕"})
    ```
    | 으로 연결하여 사용 가능

- LangGraph  
    - LangChain은 선형적인 체인 구조로, 이전 작업으로 되돌아가기 어려운 문제가 있음. 순환/반복/분기 로직을 구현하기 어려움  
    -> LangGraph를 통해 비선형적인 작업 형태로 사이클, 루프를 만들 수 있음

    - 노드: 랭체인의 컴포넌트의 역할과 비슷함.  
    - 엣지: 노드와 노드를 연결하는 선으로, 어떤 노드에서 다음 노드로 연결될지 결정. '조건부 엣지'를 통해 분기 로직을 쉽게 구현 가능  

- LangSmith: LLM 기반 어플리케이션을 위한 개발/운영 플랫폼  
    - 핵심 기능:
        1. 디버깅 및 추적: 어플의 모든 실행 과정을 단계별로 시각화
        2. 테스트 및 평가: 미리 준비된 질문&정답 셋을 만들어, 개발 중인 어플의 성능을 자동 평가  
        3. 모니터링: 사용자들이 어떻게 사용하는지 모디터링(API 호출, 지연시간, 토큰 사용량)  
        4. 프롬포트 허브: 다양한 프롬포트 저장, 버전 관리, 테스트 공간 제공

- 외부 API 활용  
LLM 내부 지식으로 해결할 수 없는 질문에 대해 외부 API를 연결하여 답변 가능
---


> ### 1-4. sLM(small Language Model)  
LLM보다 규모가 작은 자연어 처리 AI 모델  
경량화, 빠른 속도, 비용, 프라이버시 면에서 유리함!

- 가지치기  
신경망에서 중요도가 낮거나, 중복이거나, 불필요한 매개변수를 제거하는 방법으로,  
뉴런 간 연결에서의 가중치(가지치기 되면 가중치는 0이 됨), 뉴런 자체, 신경망 계층의 매개변수를 가지치기 할 수 있음

- 양자화  
고정밀(높은 비트) 데이터 -> 저정밀(int8 같은 낮은 비트) 데이터로 변환  
모델 가중치, 활성화 값 등에 양자화를 적용 -> 계산 부하를 줄이고 추론 속도 높일 수 있음

- 지식 증류  
'학생 모델'의 예측을 '교사 모델'과 일치시키고, 추론 과정도 모방하도록 학습  
-> 더 큰 모델의 지식을 작은 모델로 '증류' 시키는 방법

- sLM의 장점  
    - 프라이버시: 로컬에서 구동 가능하다면, 서버 전송과정이 없어서 보안에 강함  
    - 비용 절감: 학습, 추론 비용이 저렴  
    - 효율성 & 맞춤 분야: 특정 작업에 최적화 시키는데 강함

- sLM의 한계  
    - 편향: LLM에 전재하는 편향으로부터 학습할 수 있음  
    - 제한: LLM에 비해 광범위한 지식 기반이 부족함. 자신의 분야가 아닐 때는 성능 저하  
    - 환각  
    - 성능과 용량의 한계(파인튜닝, 지식 증류, RAG, 결합 추론(지식/성능), 양자화, 가지치기(용량)으로 보완)

- 결합 추론(sLM과 LLM의 결합)  
    - SMART 프레임워크: sLM이 풀 수 없는 부분만 선택적으로 LLM에게 도움 받는 방법  
    - 작동 과정  
        1. sLM이 문제 풀이 제안을 제작  
        2. sLM이 풀이 단계의 신뢰성을 점수로 평가  
        3. 2에서 불확실한 단계를 LLM에게 요청

    - -> 정확도 증가, 토큰 사용량 절약 가능
---

> ## 2. 생성 모델

> ### 2-1. 분류 모델 vs 생성 모델  

정답 라벨을 가지고 학습시키는 분류 모델과 다르게, 생성 모델에는 정답이 없다.  
단, 입력 데이터의 **확률 분포 P(x)를 알고**, 그 분포를 따르는 유사한 데이터를 **생성**하는 것!

생성 모델의 종류  
- 명시적 확률밀도 모델: 학습 데이터의 분포를 기반으로 생성  
    1. Tractable Density: 확률 분포를 직접적으로 표현하고 likelihood를 계산할 수 있는 경우  
    2. Approximate Density: 정확한 likelihood 계산이 어렵기에 근사하는 경우  
- 암시적 확률밀도 모델: 학습 데이터의 분포와 상관없이 생성

> ### 2-2. AE, VAE  

> AE(Auto Encoder)  
**입력과 동일한 출력**을 만드는 것을 목표로 하는 신경망  

- 차원 축소, 특징 추출, 노이즈 제거, 이상 탐지 등 데이터 복원 쪽이나 특성 학습에 많이 사용  

![오토인코더](https://blog.kakaocdn.net/dna/cDCfoH/btr16iLsgIV/AAAAAAAAAAAAAAAAAAAAAIVsx62iHRPrGCu6KTGEG949RZ3dBVTBVbyLBhDJJEe_/img.png?credential=yqXZFxpELC7KVnFOS48ylbz2pIh7yKj8&expires=1764514799&allow_ip=&allow_referer=&signature=9Fwal3hzMBrDOl7GgvDFpLNvksQ%3D)

- 인코더($g_\phi$): 고차원 입력 데이터(x)를 잠재 표현(z)로 변환하는 네트워크  
    - $\mathbf{z}=g_\phi(\mathbf{x})$  
이 때 인코더가 압축한 z를 **잠재(Latent) 벡터**라고 부르고, 이 잠재 백터에는 입력 데이터의 중요한 정보가 압축되어 있음  

-> 차원축소가 잘 되었는지 검증하기 위해 디코더가 필요  

- 디코더($f_\theta$): 잠재 표현(z)를 풀고 입력을 재복원하여, 출력(x')하는 네트워크  
    - $\mathbf{x'}=f_\theta(\mathbf{z})=f_\theta(g_\phi(\mathbf{x}))$  
    잠재 벡터를 원래대로 복원하여 잘 압축된 것인지 검증함  

- 잠재 표현(z): 원본 데이터가 저차원으로 압축된 벡터

> VAE: 확률적(Variational) 오토 인코더  
데이터를 잠재 공간으로 인코딩 -> 잠재 공간에서 다시 그 데이터를 디코딩 -> 원본 데이터와 유사한 결과 생성

- 이미지 생성, 텍스트 생성, 신호 처리, 이미지 보간 등 **새로운 데이터 생성** 쪽에 많이 사용

- AE는 인코더 학습을 위해 디코더를 붙인 것이지만,  
VAE는 디코더 학습을 위해 인코더를 붙인 구조  

- AE는 단순히 입력값을 재구성하지만,  
VAE는 잠재 코드 값을 **가우시안 확률분포에 기반한 확률값**으로 나타낸다.  

![VAE](https://gaussian37.github.io/assets/img/dl/concept/vae/0.png)

- 인코더($g_\phi$): 고차원 입력 데이터(x)를 단일 벡터(AE 방식)로 변환하는 것이 아닌, 분포(평균 $\mu$, 표준편차 $\sigma$)로 변환하는 네트워크  
    - $\epsilon$은 정규분포로부터 샘플링된 잡음(표준 정규분포 N(0,1) 에서 샘플링된, 무작위성을 제공하는 '재료')으로, 이를 이용해 Reparameterization trick을 수행.  
        - 이 $\epsilon$을 $\mu$와 $\sigma$와 결합($z = \mu + \sigma \times \epsilon$)하여,
        - '샘플링'이라는 미분 불가능한 과정을 '계산'이라는 미분 가능한 과정으로 대체하는 것  
        - -> VAE가 역전파를 통해 학습할 수 있음

    이 때 인코더가 압축한 z를 **잠재(Latent) 벡터**라고 부르고, 이 잠재 백터에는 입력 데이터의 중요한 정보가 **확률적 형태로** 압축되어 있음  

- 디코더($f_\theta$): 샘플링된 잠재 표현(z)를 풀고 입력을 **확률적 복원**하여, 출력(x')하는 네트워크  

### 2-3. GAN  
Generative Adversarial Network(생성적 적대 신경망)  
생성자 신경망과 판별자 신경망이 적대적으로 경쟁 -> 훈련 -> 더 정교해짐

- GAN의 구조  
    - 생성자의 목적: 진짜 분포에 가까운 가짜 분포 생성  
    - 판별자의 목적: 표본이 가짜 분포인지 진짜 분포인지 결정  

-> 생성자는 더 진짜같은 분포 생성, 판별자는 탐지 능력 강화  
-> 판별자가 진짜/가짜를 한쪽으로 판단하지 못하는 경계를 최적 솔루션으로 간주

- 최종 목표인 Global Optimum: 위조지폐범(생성자)이 너무 완벽한 위조지폐를 만들어서, 경찰(판별자)이 50%의 확률로밖에 구별하지 못하는 상태(즉, 무작위로 찍는 수준이 되는 균형점)

- GAN의 장점: 재현도가 높다  
- GAN의 한계  
    - 학습이 불안정(진동)  
    처음에는 학습이 잘 되다가, epoch가 커질수록 진동이 커지고, loss가 떨어지지 않음  
        - '끝없는 숨바꼭질' 문제: 생성자와 판결자가 번갈아가면서 업데이트하는 구조 때문에, Global optimum에 수렴하지 못하고 서로의 약점만 좇을 수 있음  
        - Mode Collapsing: 생성자가 다양한 데이터를 못 만들고, 자기가 잘 만드는 하나의 결과물만 계속 만들어내는 현상

### 2-4. 디퓨전 모델  
입력 이미지에 (정규 분포를 가진)노이즈를 여러 단계로 추가  
-> 여러 단계에 걸쳐 (정규 분포를 가진)노이즈를 제거  
-> 입력 이미지와 **유사한 확률 분포인** 결과 이미지 생성

![디퓨전](https://blog.kakaocdn.net/dna/cNxrba/btsytZm3X1e/AAAAAAAAAAAAAAAAAAAAAOKpd6pbpvpqyIi92rYXrBbRyVpO7xwqmvLs7VM0QdHM/img.png?credential=yqXZFxpELC7KVnFOS48ylbz2pIh7yKj8&expires=1764514799&allow_ip=&allow_referer=&signature=QgDofLCSEfOk4f9zxADeBqQaHOs%3D)
- 순확산(Forward Diffusion)  
데이터에 점진적으로 노이즈를 추가  
원래 데이터 -> 무작위 노이즈로 변경

- 역확산(Reverse Diffusion)  
노이즈 데이터에서 원본 데이터를 재구성하는 과정  
노이즈를 각 단계에서 점진적으로 제거(순확산 과정의 반대 방향) -> 원래 데이터 복원

![디퓨전 수능](https://www.hanbit.co.kr/data/cms/CMS6968709109_thumb.png)  
(수능 국어 문제에 나왔다고 하네요)